
# 🤖 AI Data Copilot — Agent com RAG + Tools

Projeto desenvolvido para demonstrar a construção de um **AI Agent com múltiplas ferramentas**, combinando:

📚 RAG (Retrieval‑Augmented Generation)  
🌐 Busca Web em tempo real  
🧮 Execução de cálculos via Python  
🧠 Raciocínio ReAct (Thought → Action → Observation)  

O objetivo é simular um **Copiloto de Dados e Operações** capaz de responder perguntas estratégicas utilizando conhecimento interno + externo.


## 🔑 Configuração da OpenAI API Key
Antes de rodar, insira sua chave abaixo (não compartilhe publicamente):

Cole sua chave aqui para testar o projeto

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "SUA_CHAVE_AQUI"

print("✅ API Key configurada!")


## 🤖 Modelo LLM
Carregando o modelo de linguagem

In [ ]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

print("🤖 Modelo carregado!")


## 📦 Imports e Dependências
Nesta etapa carregamos todas as bibliotecas utilizadas no projeto.

📚 Bibliotecas principais do projeto

In [ ]:

import os

from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.document_loaders import TextLoader, PyPDFLoader

from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

from langchain.tools import Tool
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import DuckDuckGoSearchRun

from langchain import hub
from langchain.agents import create_react_agent, AgentExecutor

print("✅ Imports carregados!")


## 📂 Carregamento dos Documentos (Base RAG)
Aqui carregamos documentos internos que servirão como base de conhecimento do Agent.

In [ ]:

print("📂 Carregando documentos...")

# 📄 Arquivos TXT
txt1 = TextLoader("churn_regras.txt", encoding="utf-8").load()
txt2 = TextLoader("faq_suporte.txt", encoding="utf-8").load()
txt3 = TextLoader("kpi_definicoes.txt", encoding="utf-8").load()

# 📑 Arquivo PDF
pdf = PyPDFLoader("sla_politica.pdf").load()

# 🔗 Unificando tudo
documents = txt1 + txt2 + txt3 + pdf

print(f"✅ Total de documentos: {len(documents)}")


## ✂️ Chunking
Dividimos os documentos em pedaços menores para melhorar a recuperação semântica.

In [ ]:

splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=100
)

chunks = splitter.split_documents(documents)

print("🧩 Total de chunks:", len(chunks))


## 🧠 Embeddings + FAISS
Transformamos os textos em vetores e criamos o índice vetorial.

In [ ]:

embeddings = OpenAIEmbeddings()

index_path = "faiss_index"

if os.path.exists(index_path):

    print("📂 Carregando índice existente...")

    vectorstore = FAISS.load_local(
        index_path,
        embeddings,
        allow_dangerous_deserialization=True
    )

else:

    print("🧠 Criando novo índice...")

    vectorstore = FAISS.from_documents(
        chunks,
        embeddings
    )

    vectorstore.save_local(index_path)

    print("✅ Índice salvo.")


## 🔎 Retriever
Responsável por buscar os trechos mais relevantes.

In [ ]:

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
)

print("🔎 Retriever criado (Top 3)")


## 🧾 Prompt Template
Define como o modelo deve usar o contexto recuperado.

In [ ]:

from langchain.prompts import PromptTemplate

template = """
Responda usando exclusivamente o texto do contexto.

- Não reescreva números.
- Não arredonde valores.
- Não complete informações.

Se não houver dado exato, diga:
"Informação não encontrada na base."

Contexto:
{context}

Pergunta:
{question}

Resposta literal:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)


## 🔗 QA Chain (RAG)
Integra Retriever + Prompt + LLM.

In [ ]:

chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=False  
)


## 🧰 Tools do Agent
Criamos ferramentas que o Agent pode utilizar.

In [ ]:
# 🧮 Python Geral
python_tool = Tool(
    name="Python Geral",
    func=python_repl.run,
    description="""
Use para cálculos matemáticos,
estatísticos ou simulações numéricas.
"""
)


# 🌐 Web Tool
search = DuckDuckGoSearchRun()

duckduckgo_tool = Tool(
    name="Busca Web",
    func=search.run,
    description="""
Use para buscar benchmarks de mercado,
métricas SaaS e melhores práticas.
"""
)


# 📚 RAG Tool
rag_tool = Tool(
    name="Base de Conhecimento",
    func=chain.run,
    description="""
Use para consultar políticas internas,
SLA, churn e definições de suporte.
"""
)

print("🧰 Tools carregadas!")


## 🤖 Criação do Agent ReAct
Aqui conectamos LLM + Tools.

In [ ]:

react_prompt = hub.pull("hwchase17/react")

agent = create_react_agent(
    llm,
    tools,
    react_prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

print("🤖 Agent pronto!")


## 🧪 Testes do Agent
Perguntas para validar cada Tool.

In [ ]:
# =========================
# 🧪 TESTES ORGANIZADOS DO AGENT
# =========================

testes = [

    # 📚 RAG
    "Qual é o SLA para prioridade crítica?",
    "Quando um cliente é considerado churn?",

    # 🌐 Web
    "Qual é o churn médio em empresas SaaS?",
    "Quais são as melhores práticas de SLA?",

    # 🧮 Python
    "Qual é 15% de 1200?",
    "Se 8% de 500 tickets estão fora do SLA, quantos são?",

    # 🔀 Multi-tools
    "Quantos tickets críticos tivemos e qual o SLA definido?",
    "Nossa satisfação está boa comparada ao mercado?"
]

for pergunta in testes:

    print("\n==============================")
    print(f"🧪 Pergunta: {pergunta}")

    resposta = agent_executor.invoke({
        "input": pergunta
    })

    print("\n🧠 Resposta:")
    print(resposta["output"])
    print("==============================\n")


# 🧪 Testes do Agent — Outputs de Resposta

Esta seção demonstra exemplos reais de perguntas feitas ao Agent e suas respectivas respostas, validando o funcionamento das Tools integradas.


## 📚 RAG — Base de Conhecimento

### 🧪 Pergunta
**Qual é o SLA para prioridade crítica?**

### 🧠 Resposta
O SLA para prioridade crítica é um tempo máximo de resolução de até **24 horas**.

---

### 🧪 Pergunta
**Quando um cliente é considerado churn?**

### 🧠 Resposta
Um cliente é considerado churn quando cancela sua assinatura ou deixa de renovar em um determinado período, contribuindo para a taxa de churn da empresa.

---

## 🌐 Web — Benchmarks de Mercado

### 🧪 Pergunta
**Qual é o churn médio em empresas SaaS?**

### 🧠 Resposta
O churn médio em empresas SaaS é idealmente abaixo de **2% ao mês**, enquanto taxas acima de **5%** podem impactar negativamente o crescimento.

---

### 🧪 Pergunta
**Quais são as melhores práticas de SLA?**

### 🧠 Resposta
As melhores práticas de SLA incluem:

- Definição clara de objetivos e metas  
- Monitoramento contínuo  
- Comunicação eficaz  
- Revisões periódicas do acordo  

---

## 🧮 Python Tool — Cálculos

### 🧪 Pergunta
**Qual é 15% de 1200?**

### 🧠 Resposta
15% de 1200 é **180**.

---

### 🧪 Pergunta
**Se 8% de 500 tickets estão fora do SLA, quantos são?**

### 🧠 Resposta
8% de 500 tickets correspondem a **40 tickets** fora do SLA.

---

## 🔀 Multi‑Tools — Raciocínio Híbrido

### 🧪 Pergunta
**Quantos tickets críticos tivemos e qual o SLA definido?**

### 🧠 Resposta
O SLA para tickets críticos é de até **24 horas para resolução**.  
(O total de tickets críticos depende da base analítica conectada.)

---

### 🧪 Pergunta
**Nossa satisfação está boa comparada ao mercado?**

### 🧠 Resposta
Para avaliar competitividade, é necessário comparar métricas como **CSAT** e **NPS** com benchmarks do setor.  
Benchmarks indicam que empresas SaaS de alto desempenho mantêm CSAT acima de **4.0/5** e NPS positivo.

---

# ✅ Conclusão dos Testes

Os testes validam que o Agent consegue:

- 📚 Consultar documentos internos (RAG)
- 🌐 Buscar benchmarks externos
- 🧮 Executar cálculos via Python
- 🔀 Combinar múltiplas fontes

Demonstrando raciocínio autônomo com Tools.